# Data Augmentation


In [82]:
import os

import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import skimage.io as io
from skimage.transform import rotate, AffineTransform, warp
from skimage.util import random_noise
from skimage.filters import gaussian

from skimage.io import imread, imsave
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from skimage.transform import rotate
from skimage.util import random_noise
from skimage.filters import gaussian
from scipy import ndimage
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision import transforms




from src.datasets import SatelliteImagesDataset, train_test_split
from src.unet import UNet
from src.path import DATA_TRAIN_IMG_PATH, MODELS_DIR, extract_archives
from src.path import DATA_TRAIN_GT_PATH

from src.plot_utils import plot_images
# from src.plot_utils import plot_image_mask

from numpy import asarray

NUM_EPOCHS = 100
MODEL_PATH = os.path.join(MODELS_DIR, 'convnet_model.pth')

from PIL import Image

Load Path

In [103]:
#Path to save Augmented Image
from src.path import DATA_DIR
AUGMENTED_IMAGES_DIR = os.path.join(DATA_DIR,'augmented/images')
AUGMENTED_GT_DIR = os.path.join(DATA_DIR,'augmented/groundtruth')

In [96]:
dataset = SatelliteImagesDataset(
        img_dir=DATA_TRAIN_IMG_PATH,gt_dir=DATA_TRAIN_GT_PATH)

image, mask = dataset[0]

#plot_images(image, mask)

type(image)

PIL.PngImagePlugin.PngImageFile

# Augmentation

In [119]:
#Enumerate Data
for i, data in enumerate(dataset):
    
    if i>2 :
        continue
    
    image, mask = data
    #Get the numpy array
    np_image = asarray(image)
    np_gt = asarray(mask)
    
    #---------Flip-------------------
    np_im_flip = np.fliplr(np_image)
    np_gt_flip = np.fliplr(np_gt)
    img_flip = Image.fromarray(np_im_flip, 'RGB')
    gt_flip = Image.fromarray(np_gt_flip, 'L')
    img_flip.save(f"{AUGMENTED_IMAGES_DIR}/satImage_Flip_{i}.png")
    gt_flip.save(f"{AUGMENTED_GT_DIR}/satImage_Flip_{i}.png")
    
    #---------Rotate-----------------
    np_im_rot = rotate(np_image,angle=45,mode = 'wrap')
    np_gt_rot = rotate(np_gt,angle=45,mode = 'wrap')
    img_rot = Image.fromarray(np_im_rot, 'RGB')
    gt_rot = Image.fromarray(np_gt_rot, 'L')
    img_rot.save(f"{AUGMENTED_IMAGES_DIR}/satImage_Rot_{i}.png")
    gt_rot.save(f"{AUGMENTED_GT_DIR}/satImage_Rot_{i}.png")
    
    #---------Random Noise------------
    np_im_noise = random_noise(np_image,var=0.2**2)
    np_gt_noise = random_noise(np_gt,var=0.2**2)
    img_noise = Image.fromarray(np_im_noise, 'RGB')
    gt_noise = Image.fromarray(np_gt_noise, 'L')
    img_noise.save(f"{AUGMENTED_IMAGES_DIR}/satImage_Noise{i}.png")
    gt_noise.save(f"{AUGMENTED_GT_DIR}/satImage_Noise{i}.png")
    
    #--------Blurr----------------------
    np_im_blurr = gaussian(np_image,sigma=1,multichannel=True)
    np_gt_blurr = gaussian(np_gt,sigma=1,multichannel=True)
    img_blurr = Image.fromarray(np_im_blurr, 'RGB')
    gt_blurr = Image.fromarray(np_gt_blurr, 'L')
    img_blurr.save(f"{AUGMENTED_IMAGES_DIR}/satImage_Blurr{i}.png")
    gt_blurr.save(f"{AUGMENTED_GT_DIR}/satImage_Blurr{i}.png")
    
    